In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/large_models", "")))

import torch
import torch.nn as nn
import torch.nn.functional as F
from loss_functions.loss import FocalLoss
from train import Model
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets

In [2]:
conf = Config()

In [3]:
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, "mlp")

X_train, X_val, X_test = X_train.unsqueeze(1), X_val.unsqueeze(1), X_test.unsqueeze(1)
print(X_train.shape)

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

test_dataset = NetworkDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, conf.batch_size)

torch.Size([356334, 1, 513])


In [ ]:
"""
class CNN(nn.Module):
    def __init__(self, input_size, channel_sizes, kernel_sizes, linear_sizes, dropout=0.2, negative_slope=0.1):
        super().__init__()

        conv_layers = []
        for (c, k) in zip(channel_sizes, kernel_sizes):
            conv_layers.append(nn.Sequential(
            nn.Conv1d(1, 32, 23),  
            nn.LeakyReLU(0.1), 
            nn.MaxPool1d(2)
        ))

        self.bn1 = nn.BatchNorm1d(size) # add batch norm

        # define linear layers
        layers = []
        in_dim = size

        for h in linear_sizes:
            layers.append(nn.Sequential(
                nn.Linear(in_dim, h),
                nn.ReLU(),
                nn.Dropout(dropout)
            ))
            in_dim = h
        
        self.linear = nn.ModuleList(layers)
        self.bn2 = nn.BatchNorm1d(in_dim)
        self.output = nn.Linear(in_dim, 24)
"""

In [9]:
class OneDCNN(nn.Module): 
    def __init__(self): 
        super(OneDCNN, self).__init__() 
        self.conv1 = nn.Sequential( nn.Conv1d(1, 32, 23), nn.LeakyReLU(0.1), nn.MaxPool1d(2) ) 
        self.conv2 = nn.Sequential( nn.Conv1d(32, 64, 19), nn.LeakyReLU(0.1), nn.MaxPool1d(2) ) 
        self.conv3 = nn.Sequential( nn.Conv1d(64, 128, 15), nn.LeakyReLU(0.1), nn.MaxPool1d(2) ) 
        self.conv4 = nn.Sequential( nn.Conv1d(128, 192, 11), nn.LeakyReLU(0.1), nn.MaxPool1d(2) ) 
        self.conv5 = nn.Sequential( nn.Conv1d(192, 256, 7), nn.LeakyReLU(0.1), nn.MaxPool1d(2) ) 
        self.flatten = nn.Flatten()
        self.bn1 = nn.BatchNorm1d(1536)
        self.fc1 = nn.Sequential( nn.Linear(1536, 512), nn.LeakyReLU(0.1), nn.Dropout(p=0.2) ) 
        self.fc2 = nn.Sequential( nn.Linear(512, 256), nn.LeakyReLU(0.1), nn.Dropout(p=0.2) ) 
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 24) 
    
    def forward(self, x): 
        x = self.conv1(x) 
        x = self.conv2(x) 
        x = self.conv3(x) 
        x = self.conv4(x) 
        x = self.conv5(x) 
        x = self.flatten(x)
        x = self.bn1(x) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        x = self.bn2(x)
        out = self.fc3(x)
        return None, out

In [10]:
model = Model(model=OneDCNN().to(conf.device),
            loss_function=FocalLoss(),
            conf=conf,
            checkpoint_path="path")

In [11]:
acc, train_loss, val_loss = model.train(train_loader, val_loader, conf.epochs)

Epoch: 1/10, Macro-F1 score: 0.71, Micro-F1 score: 0.92, Macro ROC AUC score: 0.98, Train loss: 0.195, Val loss: 0.110
Epoch: 2/10, Macro-F1 score: 0.78, Micro-F1 score: 0.95, Macro ROC AUC score: 0.99, Train loss: 0.083, Val loss: 0.076
Epoch: 3/10, Macro-F1 score: 0.81, Micro-F1 score: 0.95, Macro ROC AUC score: 0.99, Train loss: 0.063, Val loss: 0.080
Epoch: 4/10, Macro-F1 score: 0.83, Micro-F1 score: 0.96, Macro ROC AUC score: 0.99, Train loss: 0.048, Val loss: 0.064
Epoch: 5/10, Macro-F1 score: 0.84, Micro-F1 score: 0.97, Macro ROC AUC score: 1.00, Train loss: 0.038, Val loss: 0.046
Epoch: 6/10, Macro-F1 score: 0.86, Micro-F1 score: 0.97, Macro ROC AUC score: 1.00, Train loss: 0.031, Val loss: 0.082
Epoch: 7/10, Macro-F1 score: 0.89, Micro-F1 score: 0.98, Macro ROC AUC score: 1.00, Train loss: 0.025, Val loss: 0.060
Epoch: 8/10, Macro-F1 score: 0.89, Micro-F1 score: 0.97, Macro ROC AUC score: 1.00, Train loss: 0.022, Val loss: 0.043
Epoch: 9/10, Macro-F1 score: 0.90, Micro-F1 scor

In [13]:
loss, metrics = model.evaluate(test_loader)
print(f"Macro-F1 score: {metrics['f1_macro']:.2f}, Micro-F1 score: {metrics['f1_micro']:.2f}, Macro ROC AUC score: {metrics['roc_auc_macro']:.2f}")

Macro-F1 score: 0.89, Micro-F1 score: 0.98, Macro ROC AUC score: 0.99
